DAT405 Assignment 4

Noah Lanai - 9808252192 - 12h work

Carl Hjalmarsson - 9305198930 - 12h work

In [1]:
import nbdime
import pandas as pd
import tarfile
from sklearn.model_selection import train_test_split
# Import libraries for Naive Bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

Task 1:

a, b, see code below

In [2]:
# Extract the email content, decode them, and convert as dataframe
def extract_files(files, m_class):  
    rows = []
    for fname in files:
        # open the tar file
        tfile = tarfile.open(fname, 'r:bz2')
        for member in tfile.getmembers():
            f = tfile.extractfile(member)
            if f is not None:
                row = f.read()
                #get all the content of file as a row
                rows.append({'message': row.decode('latin-1'), 'class': m_class})
        tfile.close()
    #return rows
    return pd.DataFrame(rows)

#get one dataframe with all of our files
df_ham = extract_files(['./20021010_easy_ham.tar.bz2','./20021010_hard_ham.tar.bz2'], 'ham')
df_spam = extract_files(['./20021010_spam.tar.bz2'], 'spam')

hamtrain, hamtest = train_test_split(df_ham, test_size = 0.25, random_state = 0)
spamtrain, spamtest = train_test_split(df_spam, test_size = 0.25, random_state = 0)
train_data = pd.concat([hamtrain, spamtrain])
test_data = pd.concat([hamtest, spamtest])

to_replace = {'ham': 0, 'spam': 1}
train_data.replace(to_replace, inplace=True)
test_data.replace(to_replace, inplace=True)

train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [6]:
# Multinomial Naive Bayes Classifier
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_data['message']).toarray()
y_train = train_data['class']
X_test = vectorizer.transform(test_data['message']).toarray()
y_test = test_data['class']

mnb = MultinomialNB()
mnb.fit(X_train, y_train)
mnb_predictions = mnb.predict(X_test)
print("Accuracy score: ", metrics.accuracy_score(mnb_predictions, y_test))

0.9794437726723095
